# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/installation.md) before you start.

# Imports

In [ ]:
import numpy as np
import os
import sys
import tensorflow as tf

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

## Object detection imports
Here are the imports from the object detection module.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/obj_detec/models/research

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

/content/drive/My Drive/Image_processing/Paper_1/TensorAPI/models/research


##Env Setup


In [ ]:
# This is needed to display the images.
%matplotlib inline
!pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5MB 37kB/s 
     |████████████████████████████████| 512kB 44.5MB/s 
     |████████████████████████████████| 3.8MB 55.5MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=0a10e264d48e2680aaa9bb03a53ac907f65189b35b946edbcf62e76c8297df08
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement tensorflow-estimator<2.5.0,>=2.4.0rc0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing ins

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:

CKPT = '/content/drive/MyDrive/obj_detec/models/research/Training_record/Output_K136N/frozen_inference_graph.pb'

PATH_TO_LABELS = '/content/drive/MyDrive/obj_detec/models/research/Training_record/tf_record/label_map.pbtxt'

NUM_CLASSES = 50

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load a (frozen) Tensorflow model into memory.

In [ ]:
import tensorflow.compat.v1 as tf

detection_graph = tf.Graph()

with detection_graph.as_default():
    
  od_graph_def = tf.GraphDef()

  with tf.gfile.GFile(CKPT, 'rb') as fid:
        
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(category_index)

{50: {'id': 50, 'name': 'Class 50'}}


## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [ ]:
from glob import glob

In [ ]:

PATH_TO_TEST_IMAGES_DIR = '/content/drive/MyDrive/obj_detec/models/research/Dataset/test_image'

#print(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.png'))
#TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.png'))  # PNG OR JPG
TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))
print("Length of test images:", len(TEST_IMAGE_PATHS))



Length of test images: 3936


In [ ]:
print(TEST_IMAGE_PATHS[130][0:69])

/content/drive/My Drive/Image_processing/Paper_1/TensorAPI/Test/test/


In [ ]:
print(TEST_IMAGE_PATHS[0][69:])
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

1479504029848000524.jpg


In [ ]:
import time
import json
counter = 0
with detection_graph.as_default():
    time0 = time.time()
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            counter = counter + 1
            print('Image {}'.format(counter))
            filename_string = image_path[69:-4]
            try:
              image = Image.open(image_path)
              # print(image.size)
              # the array based representation of the image will be used later in order to prepare the
              # result image with boxes and labels on it.
              image_np = load_image_into_numpy_array(image)
              # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
              image_np_expanded = np.expand_dims(image_np, axis=0)

              #time0 = time.time()

              # Actual detection.
              (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})

              

              # Visualization of the results of a detection.
              # vis_util.visualize_boxes_and_labels_on_image_array(
              #  image_np,
              #  np.squeeze(boxes),
              #  np.squeeze(classes).astype(np.int32),
              #  np.squeeze(scores),
              #  category_index,
              #  use_normalized_coordinates=True,
              #  line_thickness=8)
              #plt.figure(figsize=IMAGE_SIZE)
              #plt.imshow(image_np)
              #plt.show()
              coordinates = vis_util.return_coordinates(
                          image,
                          np.squeeze(boxes), 
                          np.squeeze(classes).astype(np.int32),
                          np.squeeze(scores),
                          category_index,
                          use_normalized_coordinates=True,
                          line_thickness=8,
                          min_score_thresh=0.000001)
              #textfile = open("json/"+filename_string+".json", "a")
              #textfile.write(json.dumps(coordinates))
              #textfile.write("\n")
              #print(coordinates)
              if coordinates == []:
                  with open('/content/drive/My Drive/obj_detec/models/research/Training_record/test/output_metric/' + filename_string + '.txt','a') as text_file:
                      text_file.close()
                  
              for coordinate in coordinates:
                  #print(coordinate)
                  (xmax, xmin, ymax, ymin, accuracy, classification_num) = coordinate
                  label = classification_num
                  detection_box = (label, accuracy, xmin, ymin, xmax, ymax)
                  
                  #if label == 46:
                  #    light = 'Red'
                      
                  #if label == 47:
                  #    light = 'Yellow'
                      
                  #if label == 48:
                  #    light = 'Green'
                      
                  #if label == 49:
                  #    light = 'off'
                      
                  #print('{} {} {} {} {} {}'.format(label, accuracy, xmin, ymin, xmax, ymax))
                  
                  #if label == 46 or label == 47 or label == 48 or label == 49: 
                  with open('/content/drive/My Drive/obj_detec/models/research/Training_record/test/output_metric/' + filename_string + '.txt','a') as text_file:
                      text_file.write(str(label) + ' ' + str(accuracy) + ' ' + str(xmin) + ' ' + str(ymin) + ' ' + str(xmax) + ' ' + str(ymax) + '\n')
                      text_file.close()    
                  
                  
                  
                  #print('{} {} {} {} {} {}'.format(label, accuracy, xmin, ymin, xmax, ymax))
                  print(detection_box)
                  #if str(label) != '46' and str(label) != '47' and str(label) != '48' and str(label) != '49':
                      
                          
              time1 = time.time()

              print("Time in milliseconds", (time1 - time0) * 1000)
            except:
                  with open('/content/drive/My Drive/obj_detec/models/research/Training_record/test/output_metric/corrupt_images_new.txt','a') as text_file:
                    text_file.write(image_path + '\n')
                    text_file.close()            

                
             

Streaming output truncated to the last 5000 lines.
Image 308
(50, 0.9999807, 458, 480, 1211, 1099)
(50, 0.9997596, 1406, 522, 1717, 625)
(50, 0.9997359, 1042, 510, 1252, 616)
(50, 0.9995832, 1657, 515, 1907, 593)
(50, 0.99906653, 225, 531, 392, 617)
(50, 0.996804, 363, 542, 522, 597)
(50, 0.028169196, 1661, 511, 1796, 554)
(50, 0.022851985, 862, 511, 1109, 640)
(3, 0.005769568, 507, 422, 587, 496)
(4, 0.0046160864, 511, 418, 592, 491)
(50, 0.0024345538, 106, 519, 187, 581)
(2, 0.002204938, 507, 413, 586, 487)
(1, 0.002088325, 509, 415, 578, 493)
(5, 0.0019870882, 511, 419, 584, 491)
(50, 0.001949667, 1865, 496, 1918, 552)
(6, 0.0015992523, 506, 413, 585, 495)
(12, 0.0015768271, 504, 421, 580, 498)
(10, 0.0013858775, 515, 419, 583, 493)
(46, 0.0011020306, 519, 413, 579, 491)
(49, 0.0009869264, 508, 415, 573, 488)
Time in milliseconds 1100562.6785755157
Image 309
Image 310
(50, 0.99998605, 452, 482, 1207, 1091)
(50, 0.9999256, 1567, 531, 1878, 626)
(50, 0.99964416, 224, 567, 383, 619)
(5